In [ ]:
import numpy as np
import pandas as pd
import re

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
#Import Libraries
import matplotlib.pyplot as plt

#to ignore warnings
import warnings
warnings.filterwarnings('ignore')
# plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')

# Set the maximum number of columns to display
pd.options.display.max_columns = None
# Set the maximum number of rows to display
pd.options.display.max_rows = 10

plt.rc("figure", figsize=(22, 8))


# <p style="color:#c0a5e3; font-size:20px">Loading data ⏳</p>

In [3]:
data = pd.read_csv('skincare_products_clean.csv') 

data

,product_name,product_url,product_type,clean_ingreds,price
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20
1,CeraVe Facial Moisturising Lotion SPF 25 52ml,https://www.lookfantastic.com/cerave-facial-mo...,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00
2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,https://www.lookfantastic.com/the-ordinary-hya...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20
3,AMELIORATE Transforming Body Lotion 200ml,https://www.lookfantastic.com/ameliorate-trans...,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50
4,CeraVe Moisturising Cream 454g,https://www.lookfantastic.com/cerave-moisturis...,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00
...,...,...,...,...,...
1133,Elemis Life Elixirs Embrace Bath and Shower El...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1134,Love Boo Splendidly Soothing Bath Soak (250ml),https://www.lookfantastic.com/love-boo-splendi...,Bath Oil,"['sodium lauroyl', 'sodium cocoamphoacetate', ...",£10.99
1135,Elemis Life Elixirs Fortitude Bath and Shower ...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1136,Connock London Kukui Oil Soothing Bath & Showe...,https://www.lookfantastic.com/connock-london-k...,Bath Oil,"['capric triglyceride', 'peg-40 sorbitan perol...",£36.00


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138 entries, 0 to 1137
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1138 non-null   object
 1   product_url    1138 non-null   object
 2   product_type   1138 non-null   object
 3   clean_ingreds  1138 non-null   object
 4   price          1138 non-null   object
dtypes: object(5)
memory usage: 44.6+ KB


There are no missing values and the ingredients column was previously thoroughly cleaned. All that remains is for the ingredients to be processed into a different format that can be used in the recommendation engine. One-hot encoding will be used.

In [5]:
for i in range(len(data['clean_ingreds'])):
    data['clean_ingreds'].iloc[i] = str(data['clean_ingreds'].iloc[i]).replace('[', '').replace(']', '').replace("'", '').replace('"', '')

In [6]:
all_ingreds = []

for i in data['clean_ingreds']:
    ingreds_list = i.split(', ')
    for j in ingreds_list:
        all_ingreds.append(j)

In [7]:
all_ingreds = sorted(set(all_ingreds))
all_ingreds[0:20]

['',
 '1,10-decanediol',
 '1,2-hexanediol',
 '1,2-hexanediol ',
 '1-methylhydantoin-2-imide',
 '10-hydroxydecanoic acid',
 '2,6-dimethyl-7-octen-2-ol',
 '2-bromo-2-nitropropane-1,3-diol',
 '2-oleamido-1',
 '3-o-ethyl ascorbic acid',
 '3-octadecanediol',
 '4-t-butylcyclohexanol',
 '7-dehydrocholesterol',
 'abies alba leaf oil',
 'abies balsamea extract',
 'abies sibirica oil',
 'acacia concinna fruit extract',
 'acacia decurrens wax',
 'acacia senegal gum',
 'acacia seyal gum extract']

In [8]:
all_ingreds.remove('')
for i in range(len(all_ingreds)):
    if all_ingreds[i][-1] == ' ':
        all_ingreds[i] = all_ingreds[i][0:-1]
        
all_ingreds = sorted(set(all_ingreds))
all_ingreds[0:20]

['1,10-decanediol',
 '1,2-hexanediol',
 '1-methylhydantoin-2-imide',
 '10-hydroxydecanoic acid',
 '2,6-dimethyl-7-octen-2-ol',
 '2-bromo-2-nitropropane-1,3-diol',
 '2-oleamido-1',
 '3-o-ethyl ascorbic acid',
 '3-octadecanediol',
 '4-t-butylcyclohexanol',
 '7-dehydrocholesterol',
 'abies alba leaf oil',
 'abies balsamea extract',
 'abies sibirica oil',
 'acacia concinna fruit extract',
 'acacia decurrens wax',
 'acacia senegal gum',
 'acacia seyal gum extract',
 'acer saccharum extract',
 'acetate']

In [9]:
one_hot_list = [[0] * 0 for i in range(len(all_ingreds))]

for i in data['clean_ingreds']:
    k=0
    for j in all_ingreds:
        if j in i:
            one_hot_list[k].append(1)
        else:
            one_hot_list[k].append(0)
        k+=1
        
ingred_matrix = pd.DataFrame(one_hot_list).transpose()
ingred_matrix.columns = [sorted(set(all_ingreds))]

ingred_matrix

1,10-decanediol 1,2-hexanediol 1-methylhydantoin-2-imide  \
0                  0              0                         0   
1                  0              0                         0   
2                  0              0                         0   
3                  0              0                         0   
4                  0              0                         0   
...              ...            ...                       ...   
1133               0              0                         0   
1134               0              0                         0   
1135               0              0                         0   
1136               0              0                         0   
1137               0              0                         0   

     10-hydroxydecanoic acid 2,6-dimethyl-7-octen-2-ol  \
0                          0                         0   
1                          0                         0   
2                          0                         0   
3                          0                         0   
4                          0                         0   
...                      ...                       ...   
1133                       0                         0   
1134                       0                         0   
1135                       0                         0   
1136                       0                         0   
1137                       0                         0   

     2-bromo-2-nitropropane-1,3-diol 2-oleamido-1 3-o-ethyl ascorbic acid  \
0                                  0            0                       0   
1                                  0            0                       0   
2                                  0            0                       0   
3                                  0            0                       0   
4                                  0            0                       0   
...                              ...          ...                     ...   
1133                               0            0                       0   
1134                               0            0                       0   
1135                               0            0                       0   
1136                               0            0                       0   
1137                               0            0                       0   

     3-octadecanediol 4-t-butylcyclohexanol 7-dehydrocholesterol  \
0                   0                     0                    0   
1                   0                     0                    0   
2                   0                     0                    0   
3                   0                     0                    0   
4                   0                     0                    0   
...               ...                   ...                  ...   
1133                0                     0                    0   
1134                0                     0                    0   
1135                0                     0                    0   
1136                0                     0                    0   
1137                0                     0                    0   

     abies alba leaf oil abies balsamea extract abies sibirica oil  \
0                      0                      0                  0   
1                      0                      0                  0   
2                      0                      0                  0   
3                      0                      0                  0   
4                      0                      0                  0   
...                  ...                    ...                ...   
1133                   0                      0                  0   
1134                   0                      0                  0   
1135                   0                      0                  0   
1136                   0                      0                  0   
1137           

In [10]:
svd = TruncatedSVD(n_components=150, n_iter = 1000, random_state = 6) # firstly reduce features to 150 with truncatedSVD - this suppresses some noise
svd_features = svd.fit_transform(ingred_matrix)
tsne = TSNE(n_components = 2, n_iter = 1000000, random_state = 6) # reduce 150 features to 2 using t-SNE with exact method
tsne_features = tsne.fit_transform(svd_features)

data['X'] = tsne_features[:, 0]
data['Y'] = tsne_features[:, 1]

In [16]:
brand_list = ["111skin", "a'kin", "acorelle", "adam revolution", "aesop", "ahava", "alchimie forever",
             "algenist", "alpha-h", "ambre solaire", "ameliorate", "american crew", "anthony", "antipodes",
             "apivita", "argentum", "ark skincare", "armani", "aromatherapy associates", "aromaworks", "aromatica",
             "aurelia probiotic skincare", "aurelia skincare",
             "australian bodycare", "avant skincare", "aveda", "aveeno", "avene", "avène",
             "bakel", "balance me", "barber pro", "bareminerals", "barry m cosmetics",
             "baxter of california", "bbb london", "beautypro", "benefit", "benton", "bioderma",
             "bioeffect", "bloom & blossom", "bloom and blossom", "bobbi brown", "bondi sands", "bubble t", "bulldog", "burt's bees",
             "by terry", "carita", "caudalie", "cerave", "chantecaille", "clinique",
             "comfort zone", "connock london", "cosmetics 27", "cosrx", "cowshed", "crystal clear", 
             "cult51", "darphin", "dear, klairs", "decleor", "decléor", "dermalogica", "dhc", "doctors formula",
             "dr. brandt", "dr brandt", "dr. hauschka", "dr hauschka", "dr. jackson's", "dr.jart+", "dr. lipp",
             "dr botanicals", "dr dennis", "dr. pawpaw", "ecooking", "egyptian magic",
             "eisenberg", "elemental herbology", "elemis", "elizabeth arden", "embryolisse",
             "emma hardie", "erno laszlo", "espa", "estée lauder", "estee lauder", "eucerin",
             "eve lom", "eve rebirth", "fade out", "farmacy", "filorga", "first aid beauty", "fit", "foreo",
             "frank body", "freezeframe", "gallinée", "garnier", "gatineau", "glamglow", "goldfaden md",
             "green people", "hawkins and brimble", "holika holika", "house 99", "huxley",
             "ilapothecary", "ila-spa", "indeed labs", "inika", "instant effects", "institut esthederm", "ioma", "klorane",
             "j.one", "jack black", "james read", "jason", "jo malone london", "juice beauty", "jurlique",
             "korres", "l:a bruket", "l'oréal men expert", "l'oreal men expert", "l'oréal paris", "l'oreal paris",
             "l’oréal paris", "lab series skincare for men",
             "lancaster", "lancer skincare", "lancôme", "lancome", "lanolips", "la roche-posay", "laura mercier",
             "liftlab", "little butterfly london", "lixirskin", "liz earle", "love boo",
             "löwengrip", "lowengrip", "lumene", "mac", "madara", "mádara", "magicstripes", "magnitone london",
             "mama mio", "mancave", "manuka doctor", "mauli", "mavala", "maybelline", "medik8", "men-u", "menaji", "molton brown", "moroccanoil",
             "monu", "murad", "naobay", "nars", "natio", "natura bissé", "natura bisse",
             "neal's yard remedies", "neom", "neostrata", "neutrogena", "niod", "nip+fab", "nuxe", "nyx",
             "oh k!", "omorovicza", "origins", "ortigia fico", "oskia", "ouai", "pai ", "paula's choice", "payot",
             "perricone md", "pestle & mortar", "pestle and mortar", "peter thomas roth",
             "philosophy", "pierre fabre", "pixi", "piz buin", "polaar", "prai", "project lip",
             "radical skincare", "rapideye", "rapidlash", "real chemistry", "recipe for men",
             "ren ", "renu", "revolution beauty", "revolution skincare", "rituals", "rmk", "rodial", "roger&gallet", "salcura",
             "sanctuary spa", "sanoflore", "sarah chapman", "sea magik", "sepai",
             "shaveworks", "shea moisture", "shiseido", "skin79", "skin authority", "skinceuticals",
             "skinchemists", "skindoctors", "skin doctors", "skinny tan", "sol de janeiro", "spa magik organiks",
              "st. tropez", "starskin", "strivectin", "sukin",
             "svr", "swiss clinic", "talika", "tan-luxe", "tanorganic", "tanworx", "thalgo", "the chemistry brand",
             "the hero project", "the inkey list", "the jojoba company", "the ordinary",
             "the organic pharmacy", "the ritual of namasté", "this works", "too faced", "trilogy", "triumph and disaster",
             "ultrasun", "uppercut deluxe", "urban decay", "uriage", "verso", "vichy",
             "vida glow", "vita liberata", "wahl", "weleda", "westlab", "wilma schumann", "yes to",
             "ysl", "zelens"]
brand_list = sorted(brand_list, key=len, reverse=True)

In [17]:
data['brand'] = data['product_name'].str.lower()
k=0
for i in data['brand']:
    for j in brand_list:
        if j in i:
            data['brand'][k] = data['brand'][k].replace(i, j.title())
    k+=1
    
data

,product_name,product_url,product_type,clean_ingreds,price,X,Y,brand
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,"capric triglyceride, cetyl alcohol, propanedio...",£5.20,48.752747,269.403809,The Ordinary
1,CeraVe Facial Moisturising Lotion SPF 25 52ml,https://www.lookfantastic.com/cerave-facial-mo...,Moisturiser,"homosalate, glycerin, octocrylene, ethylhexyl,...",£13.00,171.147583,-273.490601,Cerave
2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,https://www.lookfantastic.com/the-ordinary-hya...,Moisturiser,"sodium hyaluronate, sodium hyaluronate, panthe...",£6.20,117.296555,164.496078,The Ordinary
3,AMELIORATE Transforming Body Lotion 200ml,https://www.lookfantastic.com/ameliorate-trans...,Moisturiser,"ammonium lactate, c12-15, glycerin, prunus amy...",£22.50,88.836792,-232.559753,Ameliorate
4,CeraVe Moisturising Cream 454g,https://www.lookfantastic.com/cerave-moisturis...,Moisturiser,"glycerin, cetearyl alcohol, capric triglycerid...",£16.00,165.990631,-311.095367,Cerave
...,...,...,...,...,...,...,...,...
1133,Elemis Life Elixirs Embrace Bath and Shower El...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"prunus amygdalus dulcis, tipa-laureth sulfate,...",£55.00,-282.238739,123.807365,Elemis
1134,Love Boo Splendidly Soothing Bath Soak (250ml),https://www.lookfantastic.com/love-boo-splendi...,Bath Oil,"sodium lauroyl, sodium cocoamphoacetate, cocam...",£10.99,-158.132538,-159.755325,Love Boo
1135,Elemis Life Elixirs Fortitude Bath and Shower ...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"prunus amygdalus dulcis, tipa-laureth sulfate,...",£55.00,-288.037292,123.473831,Elemis
1136,Connock London Kukui Oil Soothing Bath & Showe...,https://www.lookfantastic.com/connock-london-k...,Bath Oil,"capric triglyceride, peg-40 sorbitan peroleate...",£36.00,-152.473984,-6.070517,Connock London


In [18]:
sorted(data.brand.unique())

["A'Kin",
 'Acorelle',
 'Aesop',
 'Ahava',
 'Alchimie Forever',
 'Alpha-H',
 'Ambre Solaire',
 'Ameliorate',
 'Antipodes',
 'Apivita',
 'Ark Skincare',
 'Armani',
 'Aromatherapy Associates',
 'Aromaworks',
 'Aurelia Probiotic Skincare',
 'Aurelia Skincare',
 'Australian Bodycare',
 'Avant Skincare',
 'Aveda',
 'Aveeno',
 'Avene',
 'Avène',
 'Balance Me',
 'Barber Pro',
 'Bareminerals',
 'Bbb London',
 'Beautypro',
 'Benefit',
 'Benton',
 'Bioderma',
 'Bloom & Blossom',
 'Bloom And Blossom',
 'Bobbi Brown',
 'Bondi Sands',
 'Bubble T',
 'Bulldog',
 "Burt'S Bees",
 'By Terry',
 'Caudalie',
 'Cerave',
 'Chantecaille',
 'Clinique',
 'Comfort Zone',
 'Connock London',
 'Cosrx',
 'Cowshed',
 'Crystal Clear',
 'Darphin',
 'Dear, Klairs',
 'Decléor',
 'Dermalogica',
 'Dhc',
 'Dr Brandt',
 'Dr Dennis',
 'Dr Hauschka',
 'Dr. Brandt',
 'Dr. Hauschka',
 'Dr. Pawpaw',
 'Dr.Jart+',
 'Egyptian Magic',
 'Elemental Herbology',
 'Elemis',
 'Elizabeth Arden',
 'Embryolisse',
 'Emma Hardie',
 'Erno Laszlo

There are slight variations in some of the brand names which would cause them to be considered as separate brands. This needs to be resolved before continuing or else the brand filter wont function correctly.

In [19]:
data['brand'] = data['brand'].replace(['Aurelia Probiotic Skincare'],'Aurelia Skincare')
data['brand'] = data['brand'].replace(['Avene'],'Avène')
data['brand'] = data['brand'].replace(['Bloom And Blossom'],'Bloom & Blossom')
data['brand'] = data['brand'].replace(['Dr Brandt'],'Dr. Brandt')
data['brand'] = data['brand'].replace(['Dr Hauschka'],'Dr. Hauschka')
data['brand'] = data['brand'].replace(["L'oreal Paris", 'L’oréal Paris'], "L'oréal Paris")

# Creating the recommendation function 

**The function below recommends products by:**

- taking the name of a product as input

- only including products of the same type

- not recommending products of the same brand name

- calculating cosine similarities and returning top 5 similar products

In [ ]:
def recommender(search, num_recommendations=5):
    cs_list = []
    brands = {}
    output = []

    # Get index of searched product
    idx = data[data['product_name'] == search].index.item()

    # Convert ingredient matrix row to binary vector
    point1 = np.array(ingred_matrix.iloc[idx][1:], dtype=np.float32)

    # Get product type and brand
    prod_type = data.loc[idx, 'product_type']
    brand_search = data.loc[idx, 'brand']

    # Filter products of the same type
    data_by_type = data[data['product_type'] == prod_type].copy()

    # Compute cosine similarity using numpy
    for j, row in data_by_type.iterrows():
        point2 = np.array(ingred_matrix.iloc[j][1:], dtype=np.float32)
        cos_sim = np.dot(point1, point2) / (np.linalg.norm(point1) * np.linalg.norm(point2))
        cs_list.append(cos_sim)

    # Add similarity scores & sort by highest similarity
    data_by_type["cos_sim"] = cs_list
    data_by_type = data_by_type.sort_values("cos_sim", ascending=False)

    # Exclude the input product itself
    data_by_type = data_by_type[data_by_type["product_name"] != search]

    # Select top `num_recommendations` ensuring brand diversity
    for _, row in data_by_type.iterrows():
        brand = row["brand"]
        if brand != brand_search and brands.get(brand, 0) < 2:
            brands[brand] = brands.get(brand, 0) + 1
            output.append(row)
        if len(output) >= num_recommendations:
            break

    # Return formatted recommendations
    print(f"\033[1mRecommending products similar to {search}:\033[0m")
    return pd.DataFrame(output)[["product_name", "cos_sim"]].head(num_recommendations)


# Using function to get recommendations 

Now we will feed some product names into the above function from a variety of product types to see what recommendations we get!

In [21]:
recommender("Origins GinZing™ Energy-Boosting Tinted Moisturiser SPF40 50ml")

Recommending products similar to Origins GinZing™ Energy-Boosting Tinted Moisturiser SPF40 50ml:


,product_name,cos_sim
87,Clinique Moisture Surge SPF25 Sheertint Hydrat...,0.565322
44,Skin Doctors Sd White & Bright (50ml),0.382546
54,Clinique Moisture Surge 72-Hour Auto-Replenish...,0.372046
34,Elemis Pro-Collagen Marine Cream SPF30 50ml,0.365338
42,Estée Lauder DayWear Multi-Protection Anti-Oxi...,0.362033


In [203]:
recommender('Avène Antirougeurs Jour Redness Relief Moisturizing Protecting Cream (40ml)')

 Recommending products similar to Avène Antirougeurs Jour Redness Relief Moisturizing Protecting Cream (40ml) : 
                                          product_name   cos_sim
40           La Roche-Posay Nutritic Intense Rich 50ml  0.408956
87   Clinique Moisture Surge SPF25 Sheertint Hydrat...  0.408248
12         First Aid Beauty Ultra Repair Cream (56.7g)  0.379663
15          First Aid Beauty Ultra Repair Cream (170g)  0.379663
100  Alpha-H Daily Essential Moisturiser Spf50+ (50ml)  0.369800


(None, None)

In [22]:
recommender('Bondi Sands Everyday Liquid Gold Gradual Tanning Oil 270ml')

Recommending products similar to Bondi Sands Everyday Liquid Gold Gradual Tanning Oil 270ml:


,product_name,cos_sim
238,Face by Skinny Tan Moisturising Oil Drops 30ml,0.442627
290,Erno Laszlo Detoxifying Cleansing Oil 195ml,0.400000
269,The Chemistry Brand Glow Oil 100ml,0.353094
272,L’Oréal Paris Extraordinary Oil Sleeping Oil N...,0.345033
232,Clinique Take The Day Off Cleansing Oil 200ml,0.340168


In [23]:
recommender('Sukin Rose Hip Oil (25ml)')


Recommending products similar to Sukin Rose Hip Oil (25ml):


,product_name,cos_sim
257,Trilogy Certified Organic Rosehip Oil 45ml,1.000000
255,Trilogy Certified Organic Rosehip Oil 20ml,1.000000
231,Pai Skincare Rosehip BioRegenerate Oil 30ml,0.577350
292,Natio Ageless Rosehip Oil Cold Pressed 15ml,0.577350
259,PIXI Rose Oil Blend 30ml,0.392232


In [24]:
recommender('La Roche-Posay Anthelios Anti-Shine Sun Protection Invisible SPF50+ Face Mist 75ml')

Recommending products similar to La Roche-Posay Anthelios Anti-Shine Sun Protection Invisible SPF50+ Face Mist 75ml:


,product_name,cos_sim
306,Garnier Ambre Solaire Sensitive Hydrating Hypo...,0.901624
317,Garnier Ambre Solaire Sensitive Hypoallergenic...,0.714590
322,Lumene Nordic Hydra [Lähde] Arctic Spring Wate...,0.360119
315,Lumene Nordic Hydra Lähde Arctic Spring Water ...,0.341704
337,Revolution Skincare x Jake Jamie Tropical Quen...,0.328196


In [25]:
recommender('Clinique Even Better Clinical Radical Dark Spot Corrector + Interrupter 30ml')

Recommending products similar to Clinique Even Better Clinical Radical Dark Spot Corrector + Interrupter 30ml:


,product_name,cos_sim
219,Darphin Dark Circle Relief and De-Puffing Eye ...,0.887904
180,Estée Lauder Idealist Pore Minimizing Skin Ref...,0.500390
223,Elizabeth Arden Prevage Advanced Daily Serum,0.453962
154,Estée Lauder Perfectionist Pro Multi-Defense A...,0.434549
177,Lancôme Génifique Double Drop Serum 20ml,0.393573


In [26]:
import joblib

# Save the ingredient matrix
joblib.dump(ingred_matrix, "ingred_matrix.pkl")

# Save the product data
data.to_csv("products.csv", index=False)

print("Model saved successfully!")


Model saved successfully!


In [30]:
def evaluate_model(num_samples=10, top_k=5):
    correct_recommendations = 0
    total_recommendations = 0

    sample_products = data.sample(num_samples)['product_name'].tolist()

    for product in sample_products:
        recommendations = recommender(product, num_recommendations=top_k)
        recommended_types = data.set_index('product_name').loc[recommendations['product_name']]['product_type'].values
        true_type = data[data["product_name"] == product]["product_type"].values[0]

        correct_recommendations += sum(recommended_types == true_type)
        total_recommendations += top_k

    precision = correct_recommendations / total_recommendations
    recall = correct_recommendations / (num_samples * top_k)

    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")

# Run Evaluation
evaluate_model()


Recommending products similar to La Roche-Posay Hyalu B5 Eyes 15ml:
Recommending products similar to The Ordinary Squalane Cleanser 50ml:
Recommending products similar to Frank Body Original Coffee Scrub 200g:
Recommending products similar to Garnier Moisture Bomb Green Tea Hydrating Face Sheet Mask for Combination Skin 32g:
Recommending products similar to Bloom & Blossom George's Candy Floss & Marshmallow Handwash:
Recommending products similar to ManCave Sensitive Body Wash 200ml:
Recommending products similar to Acorelle Pure Harvest Body Perfume - 100ml:
Recommending products similar to Garnier Ultralift Anti Ageing Radiance Boosting Face Sheet Mask 32g:
Recommending products similar to ilapothecary Powerful Purifying Face Scrub 50g:
Recommending products similar to The Ritual of Namasté Skin Brightening Face Exfoliator:
Precision: 1.00, Recall: 1.00


In [209]:
recommender('Garnier Organic Argan Mist 150ml')

 Recommending products similar to Garnier Organic Argan Mist 150ml : 
                                          product_name   cos_sim
378   The Ritual of Namasté Urban Hydrating Mist 100ml  0.526137
315  Lumene Nordic Hydra Lähde Arctic Spring Water ...  0.435011
322  Lumene Nordic Hydra [Lähde] Arctic Spring Wate...  0.429801
379         Acorelle Pure Harvest Body Perfume - 100ml  0.415227
307   La Roche-Posay Toleriane Ultra 8 Face Mist 100ml  0.412021


(None, None)

In [210]:
recommender('Shea Moisture 100% Virgin Coconut Oil Daily Hydration Body Wash 384ml')

 Recommending products similar to Shea Moisture 100% Virgin Coconut Oil Daily Hydration Body Wash 384ml : 
                                           product_name   cos_sim
1093            Caudalie Rose de Vigne Shower Gel 200ml  0.494913
1077           Caudalie Thé des Vignes Shower Gel 200ml  0.494913
1041  Pai Skincare Gentle Genius Camellia and Bergam...  0.393398
1091       Laura Mercier Almond Coconut Body Wash 200ml  0.353553
1023      Aveeno Baby Daily Care Hair & Body Wash 500ml  0.353009


(None, None)

In [31]:
recommender('JASON Soothing Aloe Vera Body Wash 887ml')

Recommending products similar to JASON Soothing Aloe Vera Body Wash 887ml:


,product_name,cos_sim
985,Bulldog Lemon & Bergamot Shower Gel 500ml,0.551039
1001,Bulldog Black Pepper & Vetiver Shower Gel 500ml,0.551039
1016,Aveeno Baby Daily Care Baby Gentle Wash 500ml,0.512806
1051,NEOM Tranquillity Body & Hand Wash 250ml,0.505496
1042,REN Clean Skincare Atlantic Kelp and Magnesium...,0.461538
